## <b>1 <span style='color:#2ae4f5'>|</span> Bibliotecas e Imports</b>

In [10]:
import math
import os
os.environ['MPLBACKEND'] = 'macosx'

from dotenv import load_dotenv
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister,transpile
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
import matplotlib.pyplot as plt

## <b>2 <span style='color:#2ae4f5'>|</span> Funções Auxiliares </b>

In [2]:
def create_oracle(qc, reg_in, reg_out, results):
    for i, result in enumerate(results):
        if result:
            qc.mcx(reg_in, reg_out, ctrl_state=i)


def create_diffusor(qc, reg_in):
    qc.h(reg_in)
    qc.x(reg_in)

    # MCZ
    qc.h(reg_in[-1])
    qc.mcx(reg_in[:-1], reg_in[-1])
    qc.h(reg_in[-1])

    qc.x(reg_in)
    qc.h(reg_in)

## <b>3 <span style='color:#2ae4f5'>|</span> String Matching </b>

### Implementação

In [6]:
def create_string_match(string_c, string_m):
    if len(string_m) > len(string_c):
        raise ValueError("String match is bigger than string to compare")
    results = [
        1 if string_c[i : i + len(string_m)] == string_m else 0 for i in range(len(string_c) - len(string_m) + 1)
    ]

    n_qubits_in = math.ceil(math.log2(len(results)))
    if n_qubits_in < 4:
        raise ValueError("Experiment with less than 5 qubits")
    print(f"Experiment with {n_qubits_in + 1} qubits")
    M = sum(results)
    N = 2**n_qubits_in

    if M == 0:
        raise ValueError("Empty oracle")

    theta = 2 * math.asin(math.sqrt(M / N))
    n_rep_without_round = (math.pi / theta - 1) / 2
    n_rep = round(n_rep_without_round)
    print("Exact number of repetitions: ", round(n_rep_without_round, 4))
    print("Number of repetitions rounded: ", n_rep)
    reg_in = QuantumRegister(n_qubits_in)
    c_reg_in = ClassicalRegister(n_qubits_in)
    reg_out = QuantumRegister(1)

    qc = QuantumCircuit(reg_in, reg_out, c_reg_in)
    qc.h(reg_in)

    qc.x(reg_out)
    qc.h(reg_out)
    for _ in range(n_rep):
        create_oracle(qc, reg_in, reg_out, results)
        create_diffusor(qc, reg_in)

    qc.measure(reg_in, c_reg_in)

    return qc

### Análises

#### Simulador Ideal

In [ ]:
string_comparison = "Eu como banana e maçã pela tarde, e outra banana de noite!"
string_match = "banana"

qc = create_string_match(string_comparison, string_match)

backend_ideal = AerSimulator()
t_qc = transpile(qc, backend_ideal)

result = backend_ideal.run(t_qc, shots=1024).result()
counts = result.get_counts()

print("Resultados:", counts)
plot_histogram(counts)
plt.title("Histogram (Ideal Simulation)")
plt.tight_layout()
plt.show()

Experiment with 7 qubits
Exact number of repetitions:  3.9195
Number of repetitions rounded:  4
Resultados: {'101010': 551, '001000': 473}


In [13]:
binario = ["101010", "001000"]

decimais = [int(b, 2) for b in binario]

print(decimais)

[42, 8]


#### Simulador com Ruido

In [14]:
#load_dotenv()

string_comparison = "Eu como banana e maçã pela tarde, e outra banana de noite!"
string_match = "banana"

qc = create_string_match(string_comparison, string_match)
service = QiskitRuntimeService(channel=os.getenv("IBM_QUANTUM_SERVICE"))

for backend in service.backends():
    if backend.name == "ibm_marrakesh":
        continue
    print("Backend: ", backend.name)
    noise_model = NoiseModel.from_backend(backend)

    backend_noisy = AerSimulator(noise_model=noise_model)
    pass_manager = generate_preset_pass_manager(
        optimization_level=3,
        backend=backend_noisy,
        layout_method="sabre",
        routing_method="sabre",
    )
    qc_best = pass_manager.run(qc)
    result = backend_noisy.run(qc_best, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.title(f"Histogram for backend {backend.name}")
    plt.tight_layout()
    plt.show()


Experiment with 7 qubits
Exact number of repetitions:  3.9195
Number of repetitions rounded:  4


qiskit_runtime_service.__init__:WARNING:2025-11-24 09:07:03,240: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Projeto-Algo_I. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-24 09:07:03,241: Loading instance: Projeto-Algo_I, plan: open


Backend:  ibm_fez
Backend:  ibm_torino
